In [3]:
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words('indonesian') + list(punctuation)

## Import dataset

In [4]:
df = pd.read_csv("data/kompas.csv")
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


## Extract BoW

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
bow = CountVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_indo, min_df=5)

In [7]:
bow_matrix = bow.fit_transform(df['teks'])

D:\MiniConda\envs\jcop_usl\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


## Topic Modeling

In [8]:
vocab = bow.get_feature_names()

## Latent semantic analysis (LSA)

In [9]:
from sklearn.decomposition import TruncatedSVD

In [10]:
lsa = TruncatedSVD(n_components=10, n_iter=10, random_state=42)

In [12]:
lsa_matrix = lsa.fit_transform(bow_matrix)

In [13]:
print(bow_matrix.shape) # hidden layer (seperti di deep learning)
print(lsa_matrix.shape) # weight / code
print(lsa.components_.shape) # fitur / topic

(2008, 25132)
(2008, 10)
(10, 25132)


In [19]:
def get_topic(model):
    return [[vocab[idx] for idx in reversed(comp.argsort()[-6:]) if vocab[idx].isalnum()] for comp in model.components_]

In [20]:
get_topic(lsa)

[['presiden', 'indonesia', 'pemerintah', 'dpr'],
 ['presiden', 'dpr', 'ketua', 'partai', 'mpr', 'tandjung'],
 ['pemerintah', 'rp', 'indonesia', 'bank', 'persen', 'utang'],
 ['rp', 'tandjung', 'dana', 'bulog', 'hukum', 'harga'],
 ['presiden', 'air', 'banjir', 'harga', 'rp', 'dpr'],
 ['harga', 'beras', 'rp', 'bbm'],
 ['mpr', 'konstitusi', 'bppn', 'uud'],
 ['indonesia', 'mpr', 'konstitusi', 'uud', 'perubahan', '1945'],
 ['pemerintah', 'dpr', 'israel', 'bppn', 'kota', 'aceh'],
 ['massa', 'rupiah', 'bunga', 'mpr', 'bank', 'suku']]

## Latent dirichlet Allocation (LDA)

In [21]:
from sklearn.decomposition import LatentDirichletAllocation

In [22]:
lda = LatentDirichletAllocation(n_components=10, max_iter=10, random_state=42)

In [23]:
lda_matrix = lda.fit_transform(bow_matrix)

In [24]:
get_topic(lda)

[['hukum', 'tandjung', 'agung', 'tim'],
 ['rumah', 'polisi', 'orang', 'jakarta'],
 ['pemerintah', 'rp', 'bppn', 'utang'],
 ['presiden', 'dpr', 'ketua', 'politik'],
 ['israel', 'aceh', 'keamanan', 'pemerintah'],
 ['indonesia', 'rp', 'dollar', 'as'],
 ['kota', 'warga', 'orang', 'korban'],
 ['indonesia', 'beras', 'harga', 'petani'],
 ['banjir', 'jakarta', 'air', 'jalan'],
 ['pesawat', 'pt', 'bca', 'ka']]